In [11]:
import pandas as pd
import numpy as np
from collections import Counter
import random
random.seed(42)

#### Code for sampling to 3 examples per tweet

In [28]:
for label_to_class in ["hate.speech", "offensive.language"]:
  for version in ["A","B","C","D","E"]:
    csv = pd.read_csv('../data/version'+version+'_train_s.csv',header=0)
    # create a new csv df
    csv_new = pd.DataFrame(csv, columns=[label_to_class, "tweet_hashed"])
    # drop all rows that have any NaN values
    csv_new_clean = csv_new.dropna(axis=0,how="any")
    # save tweets into a list
    tweet=list(csv_new_clean['tweet_hashed'])
    # count tweet freqs
    tweet_count=Counter(tweet)
    for t, c in tweet_count.items():
        if c > 3:
           # get the index for a specific tweet into a list
           index = csv_new_clean[csv_new_clean.tweet_hashed == t].index.tolist()
           # randomly choose index to drop
           index_to_drop = random.sample(index, c-3) # drop to 3 samples per tweet
           csv_new_clean = csv_new_clean.drop(index_to_drop, axis=0)
        elif c < 3:
           index = csv_new_clean[csv_new_clean.tweet_hashed == t].index.tolist()
           csv_new_clean = csv_new_clean.drop(index, axis=0)
    csv_new_clean.to_csv("../data_sampled/version"+version+label_to_class+'_train_sampled.csv')

#### Code for getting the median label for each tweet

In [ ]:
def get_median(data):
    data = sorted(data)
    size = len(data)
    if size % 2 == 0:
        median = (data[size//2]+data[size//2-1])/2
        data[0] = median
    if size % 2 == 1:
        median = data[(size-1)//2]
        data[0] = median
    return data[0]
for label_to_class in ["hate.speech", "offensive.language"]:
  for version in ["A","B","C","D","E"]:
    new_csv = pd.read_csv("../temp/version"+version+label_to_class+'_train_sampled.csv',header=0)
    median_csv= pd.DataFrame(columns=[label_to_class, "tweet_hashed"])
    for tweet in list(set(list(new_csv["tweet_hashed"]))):
        l = []
        labels=list(new_csv[label_to_class])
        index = new_csv[new_csv.tweet_hashed == tweet].index.tolist()
        for i in index:
            l.append(labels[i])
        median = get_median(l)
        median_csv.loc[len(median_csv)]=[median, tweet]
    median_csv.to_csv("../data_median/version"+version+label_to_class+'_train_median.csv')